In [39]:
from langchain_groq import ChatGroq

In [56]:
llm=ChatGroq(model="llama-3.1-8b-instant")

In [10]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader(r"C:\Users\Rahul\OneDrive\Desktop\Document_Portal\notebook\interview-questions.pdf")
doc = loader.load()
len(doc)

6

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_documents(doc)
chunks

[Document(metadata={'producer': 'Skia/PDF m143 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'interview-questions', 'source': 'C:\\Users\\Rahul\\OneDrive\\Desktop\\Document_Portal\\notebook\\interview-questions.pdf', 'total_pages': 6, 'page': 0, 'page_label': '1'}, page_content='Embeddings,  Vector  DB,  Retrieval,  and  Similarity:  ●  What  embedding  and  vector  DB  did  you  use  and  why?  ●  What  was  the  vector  size  and  what  is  the  impact  of  vector  length?  ●  Which  vector  DB  did  you  use  and  why?  ●  What  are  different  types  of  similarity  search  (cosine,  Euclidean,  Manhattan)  and  when  to  \nuse\n \nwhat?\n ●  How  to  perform  retrieval  operation?  ●   How  do  you  handle  metadata  in  vector  DB?  ●  What  is  a  vector  DB?  \nRAG  (Retrieval-Augmented  Generation):  ●   What  is  RAG  architecture?  ●  How  does  RAG  work?  ●   What  are  RAG  failures,  and  how  do  you  evaluate  RAG?  ●   Where  does  the  evalu

In [20]:
import os
from dotenv import load_dotenv
load_dotenv()
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
Gemini_API_KEY=os.getenv("Gemini_API_KEY")
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")
HuggingFace_API_KEY=os.getenv("HuggingFace_API_KEY")
GROQ_API_KEY=os.getenv("GROQ_API_KEY")

In [29]:
# ...existing code...
import os
# ensure token is available to huggingface-hub as HUGGINGFACEHUB_API_TOKEN
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HuggingFace_API_KEY

from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2"
)
# ...existing code...

c:\Users\Rahul\OneDrive\Desktop\Document_Portal\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Rahul\OneDrive\Desktop\Document_Portal\env\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Rahul\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Pyt

In [ ]:
from langchain.vectorstores import FAISS

vectorstore = FAISS.from_documents(chunks, embedding_model)


In [31]:
from langchain.prompts import PromptTemplate
prompt_template = """
        Answer the question based on the context provided below. 
        If the context does not contain sufficient information, respond with: 
        "I do not have enough information about this."

        Context: {context}

        Question: {question}

        Answer:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [35]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

In [36]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

In [37]:
from langchain_core.runnables import RunnablePassthrough

In [53]:
def format_docs(doc):
    return "\n\n".join([lines.page_content for lines in doc])

In [57]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | PROMPT
    | llm
    | StrOutputParser()
)

In [58]:
rag_chain.invoke("What is this document is really about?")

"This document appears to be a discussion about a Generative AI project, including its technical details, implementation, and challenges. It covers various aspects of the project, such as the use of Large Language Models (LLMs), vector databases, and cloud services, as well as the integration of Responsible AI principles, Human-in-the-Loop, and other techniques to improve quality and trust.\n\nThe document also touches on topics like LLM understanding and comparison, model performance, accuracy, and retraining, as well as conversational AI, tech stack, and infrastructure integration. It seems to be a comprehensive overview of the project's architecture, design, and implementation.\n\nOverall, this document is likely a documentation of a research project or a technical discussion about a Generative AI project, possibly for a academic or industry setting."